# CIRCE vs Semantic Search - ATECO 2022

In [1]:
import pandas as pd
import numpy as np
import ast
import requests
import time
import json
from typing import List
from semantic_search.data import build_corpus
from semantic_search.local import LocalKnowledgeBase

/anaconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configs

In [28]:
EXTRACT_CIRCE : bool = False
CIRCE_URL     : str  = "https://www.istat.it/wp-content/themes/EGPbs5-child/ateco/atecor.php"
CIRCE_FILENAME: str  = "data/circe_results.json"
MERGE_DESCS   : bool = False

MODEL_ID : str = "BAAI/bge-m3"  # paraphrase-multilingual-MiniLM-L12-v2, LaBSE, Qwen/Qwen3-Embedding-0.6B

---

## Data

In [24]:
ateco_df = pd.read_csv("data/ateco_2022_raw.csv")
ateco_df = ateco_df[ateco_df["section"].str.len() == 8]

codes = []
texts = []
for i, row in ateco_df.iterrows():
    if row["title"]:
        codes.append(row["section"])
        texts.append(row["title"].lower())

    if type(row["description"]) == str:
        desc = row["description"]
        desc = desc.split(". Sono escluse")[0]
        desc_list = desc.split(" - ")
        for d in desc_list:
            codes.append(row["section"])
            texts.append(d.lower().strip("- "))

if MERGE_DESCS:
    ateco_df_merge = pd.DataFrame({
        "code": codes,
        "text": texts
    })
    ateco_df_merge = ateco_df_merge.groupby("code").aggregate({
        "text": lambda x: ".\n".join(x)
    })
    codes = ateco_df_merge.index.tolist()
    texts = ateco_df_merge["text"].tolist()

sample_queries_df = pd.read_csv("data/ateco_sample_queries.csv", sep=";")
sampled_df = sample_queries_df.sample(1000, random_state=42)
sampled_df.drop(columns=["ID"], inplace=True)
sampled_df.rename(columns={"Stringa": "query"}, inplace=True)

queries = sampled_df["query"].tolist()

Import CIRCE labeled data.

In [25]:
if EXTRACT_CIRCE:
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    with open(CIRCE_FILENAME, "w") as f:
        json.dump({}, f)

    results_dict = {}
    for idx, query in enumerate(queries):
        results_dict[idx] = {}
        results_dict[idx]["query"] = query
        data = {"search": query}
        try:
            response = requests.post(CIRCE_URL, data=data, headers=headers)
            response_dict = ast.literal_eval(response.text.replace('""', '"'))

            results_dict[idx]["result"] = {}

            for j, res in enumerate(response_dict["0"]):
                results_dict[idx]["result"][j] = {}
                code = res["ateco_code"]
                desc = res["ateco_description"]
                results_dict[idx]["result"][j]["code"] = code
                results_dict[idx]["result"][j]["desc"] = desc

        except:
            results_dict[idx]["result"] = "ERROR"

        with open(CIRCE_FILENAME, "r") as f:
            json_data = json.load(f)
        json_data.update(results_dict)
        with open(CIRCE_FILENAME, "w") as f:
            json.dump(json_data, f)

        time.sleep(0.005)

else:
    with open(CIRCE_FILENAME, "r") as f:
        results_dict = json.load(f)

---

## Semantic Search
First, we create the knowledge base for the ATECO 2022 classification.

In [26]:
corpus = build_corpus(
    texts=texts,
    ids=list(range(len(texts))),
    metadata=[{"code": c} for c in codes]
)
base = LocalKnowledgeBase(
    corpus=corpus,
    model_id="BAAI/bge-m3",
    batch_size=16
)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [00:18<00:00, 12.97it/s]


Then, we search the knowledge base and extract the matching between CIRCE and semantic search.

In [33]:
match_top_k = 10

results = base.search(queries, top_k=30)

def parse_retrieved(results):
    codes = [r.metadata["code"] for r in results]
    texts = [r.text for r in results]
    scores = [r.score for r in results]

    df = pd.DataFrame({
        "code": codes,
        "matched_text": texts,
        "score": scores
    })

    grouped_df = df.groupby("code").aggregate({
        "score": "max",
    }).reset_index()

    return grouped_df.sort_values("score", ascending=False)

queries_filtered = []
top_results = []
circe_guess = []
sem_search_guess = []

for idx, res in enumerate(results):
    circe = results_dict[str(idx)]["result"]
    if circe == "ERROR":
        continue
    ateco_circe = set(np.unique([circe[key]["code"] for key in circe.keys()]).tolist())

    sem_search_res = parse_retrieved(res.results).iloc[:match_top_k]["code"].tolist()
    sem_search_res = set([r[:-1] for r in sem_search_res])

    if ateco_circe.intersection(sem_search_res):
        top_results.append(1)
    else:
        top_results.append(0)

    circe_guess.append(list(ateco_circe))
    sem_search_guess.append(list(sem_search_res))
    queries_filtered.append(queries[idx])

print(f"Overlap (Top {match_top_k})): {round(np.mean(top_results)*100, 2)}%")

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 23.06it/s]


Overlap (Top 10)): 66.94%


In [44]:
export_df = pd.DataFrame({
    "QUERY": queries_filtered,
    "CIRCE": circe_guess,
    "SEMANTIC": sem_search_guess,
    "OVERLAP": top_results
})

export_df.to_csv("data/circe_vs_semantic.csv", index=False)

,QUERY,CIRCE,SEMANTIC,OVERLAP
0,creazione oggetti casa,"[47.59.2, 46.44.3]","[43.99.0, 43.91.0, 23.13.0, 32.99.3, 22.29.0, ...",0
1,"il commercio, anche per conto terzi ed anche i...",[45.11.0],"[46.14.0, 79.11.0, 45.31.0, 45.11.0, 66.12.0, ...",1
2,l attivita svolta e commercio al dettagli...,[47.42.0],"[46.52.0, 47.63.0, 47.41.0, 47.43.0, 47.54.0, ...",1
3,fabbricazione di birra,[11.05.0],"[10.42.0, 11.05.0, 56.30.0, 23.14.0, 20.14.0, ...",1
4,salone di estetica,[96.02.0],"[20.42.0, 86.23.0, 45.20.3, 86.22.0, 59.20.3, ...",1
...,...,...,...,...
987,impresa edilizia costruzione demolizione,"[43.11.0, 42.11.0, 41.20.0]","[43.11.0, 38.31.1, 39.00.0, 42.91.0, 43.91.0, ...",1
988,consulente attivita intellettuale,[n.c. ],"[86.90.3, 66.19.2, 69.10.1, 94.12.2, 62.02.0, ...",0
989,servizi consulenza aziendale marketing,[70.22.0],"[66.19.2, 73.11.0, 69.10.1, 70.22.0, 62.02.0, ...",1
990,progettazione e sviluppo in ambito tecnologico,"[74.10.3, 62.01.0, n.c. , 71.12.1]","[71.20.2, 62.01.0, 71.20.1, 62.02.0, 71.12.1, ...",1
